In [36]:
from pypdf import PdfReader, PdfWriter, PageObject, Transformation
import camelot
import pandas as pd


In [37]:
# Split PDF into pages and save a new PDF with selected pages

# Define input and output paths

input_path = r"C:\Users\alex\coding_projects\bls_data\data\pdf\bls_1312-10_1976.pdf"
output_path = r"C:\Users\alex\coding_projects\bls_data\data\pdf\bls_1312-10_1976_manufacturing_test.pdf"

# Choose your page range (e.g., pages 3 to 7)
start_page = 50   # inclusive, 1-based
end_page = 52     # inclusive, 1-based

# Read the PDF
reader = PdfReader(input_path)
writer = PdfWriter()

# Add chosen pages (PyPDF2 uses 0-based indexing internally)
for page_num in range(start_page - 1, end_page):
    writer.add_page(reader.pages[page_num])

# Save the new PDF
with open(output_path, "wb") as f:
    writer.write(f)

print(f"Saved new PDF to {output_path} with pages {start_page}-{end_page}")


Saved new PDF to C:\Users\alex\coding_projects\bls_data\data\pdf\bls_1312-10_1976_manufacturing_test.pdf with pages 50-52


In [38]:
# Crop odd and even pages differently in a PDF

# Define input and output paths
input_path = r"C:\Users\alex\coding_projects\bls_data\data\pdf\bls_1312-10_1976_manufacturing_test.pdf"
output_path = r"C:\Users\alex\coding_projects\bls_data\data\pdf\bls_1312-10_1976_cropped_odd_even.pdf"

# Read the PDF
reader = PdfReader(input_path)
writer = PdfWriter()

# Define crop dimensions for odd and even pages (in points, 1 inch = 72 points)
odd_crop = {"left": 0, "bottom": 87, "right": 0, "top": 125}  # Odd pages crop
even_crop = {"left": 0, "bottom": 87, "right": 0, "top": 125}  # Even pages crop

for i, page in enumerate(reader.pages):
    # Get the current mediabox
    mediabox = page.mediabox

    # Determine crop dimensions based on page index (0-based)
    crop = odd_crop if i % 2 == 0 else even_crop

    # Adjust the mediabox to crop the page
    mediabox.lower_left = (mediabox.lower_left[0] + crop["left"], mediabox.lower_left[1] + crop["bottom"])
    mediabox.upper_right = (mediabox.upper_right[0] - crop["right"], mediabox.upper_right[1] - crop["top"])

    # Add the cropped page to the writer
    writer.add_page(page)

# Save the cropped PDF
with open(output_path, "wb") as f:
    writer.write(f)

print(f"Cropped PDF saved to {output_path}")

Cropped PDF saved to C:\Users\alex\coding_projects\bls_data\data\pdf\bls_1312-10_1976_cropped_odd_even.pdf


In [39]:
input_path = r"C:\Users\alex\coding_projects\bls_data\data\pdf\bls_1312-10_1976_manufacturing_test.pdf"
output_path = r"C:\Users\alex\coding_projects\bls_data\data\pdf\bls_1312-10_1976_manufacturing_split_test.pdf"

reader = PdfReader(input_path)
writer = PdfWriter()

for page in reader.pages:
    width = float(page.mediabox.width)
    height = float(page.mediabox.height)

    # Left half page
    left_page = PageObject.create_blank_page(width=width / 2, height=height)
    left_page.merge_page(page)
    left_page.mediabox.upper_right = (width / 2, height)
    writer.add_page(left_page)

    # Right half page: create blank page, merge original page shifted left by half width
    right_page = PageObject.create_blank_page(width=width / 2, height=height)

    # Translate content left by half the width (so right half fits)
    translation = Transformation().translate(-width / 2, 0)
    right_page.merge_transformed_page(page, translation)
    right_page.mediabox.upper_right = (width / 2, height)
    writer.add_page(right_page)

with open(output_path, "wb") as f:
    writer.write(f)

print(f"Saved split PDF to {output_path}")


Saved split PDF to C:\Users\alex\coding_projects\bls_data\data\pdf\bls_1312-10_1976_manufacturing_split_test.pdf


In [40]:
input_pdf = r"C:\Users\alex\coding_projects\bls_data\data\pdf\bls_1312-10_1976_manufacturing_split_test.pdf"
output_csv = r"C:\Users\alex\coding_projects\bls_data\data\pdf\bls_1312-10_1976_manufacturing_split_test.csv"

# Extract tables from all pages using stream mode
tables = camelot.read_pdf(input_pdf, pages='all', flavor='stream', strip_text='\n')

print(f"Found {tables.n} tables")

# Combine all tables into a single DataFrame
df_list = [table.df for table in tables]
combined_df = pd.concat(df_list, ignore_index=True)

# Optional: reset index, drop empty columns, or clean headers here if needed
combined_df.to_csv(output_csv, index=False)

print(f"Saved combined CSV to {output_csv}")


Found 8 tables
Saved combined CSV to C:\Users\alex\coding_projects\bls_data\data\pdf\bls_1312-10_1976_manufacturing_split_test.csv


C:\Users\alex\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\camelot\parsers\base.py:238: UserWarning: No tables found in table area (0, 0, 378.25, 576.0)
  cols, rows, v_s, h_s = self._generate_columns_and_rows(bbox, user_cols)
C:\Users\alex\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\camelot\parsers\base.py:238: UserWarning: No tables found in table area (0, 0, 378.25, 576.0)
  cols, rows, v_s, h_s = self._generate_columns_and_rows(bbox, user_cols)
